In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
prefix = os.getcwd().split("/src")[0]
src_dir = os.path.join(prefix, "src")
data_dir = os.path.join(prefix, "dataset")
sys.path.append(src_dir)

In [2]:

import torch
import mlflow
import math
from torch import nn
from models import VisuelleDataset
from utils import misc_utils
from torch import optim
from tqdm import tqdm

/Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
vis_dataset = VisuelleDataset(data_dir)

2024-02-28 22:48:17.463 | INFO     | models.visuelle_dataset:__init__:32 - ====== DATASET ======
2024-02-28 22:48:17.464 | INFO     | models.visuelle_dataset:__init__:33 - Train path: /Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/dataset/train.csv
2024-02-28 22:48:17.464 | INFO     | models.visuelle_dataset:__init__:34 - Test path: /Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/dataset/test.csv
2024-02-28 22:48:17.465 | INFO     | models.visuelle_dataset:__init__:35 - Category path: /Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/dataset/category_labels.pt
2024-02-28 22:48:17.465 | INFO     | models.visuelle_dataset:__init__:36 - Color path: /Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/dataset/color_labels.pt
2024-02-28 22:48:17.465 | INFO     | models.visuelle_dataset:__init__:37 - Fabric path: /Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/dataset/fabric_labels.pt


In [4]:
train_loader = vis_dataset.get_data_loader(train=True, batch_size=64)
test_loader = vis_dataset.get_data_loader(train=False, batch_size=64)

100%|##########| 497/497 [00:00<00:00, 821.05it/s]


In [6]:
targets, attrs, temporals, gtrends, vtrends = next(iter(train_loader))

In [5]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [7]:
# class TransformerPoolingForecast(nn.Module):
#     def __init__(self, num_trends: int=4):
#         super(TransformerPoolingForecast, self).__init__()
#         self.encoder = nn.TransformerEncoder(
#             nn.TransformerEncoderLayer(d_model=num_trends, nhead=2, dim_feedforward=256), 
#             num_layers=1
#         )
#         self.pooling = nn.AdaptiveAvgPool2d((1, 32))
#         self.linear = nn.Linear(32, 12)
#         self.criterion = nn.MSELoss()
#         self.optimizer = optim.Adam(self.parameters(), lr=0.1)

#     def forward(self, x, y):
#         ### Expect x shape (batch_size, num_trends, trend_len)
#         x = self.encoder(x.permute(0, 2, 1))
#         x = self.pooling(x)
#         x = self.linear(x)
#         x = x.view(-1, 12)
#         loss = self.criterion(y, x)
#         return loss
    
#     def predict(self, x):
#         x = self.encoder(x.permute(0, 2, 1))
#         x = self.pooling(x)
#         x = self.linear(x)
#         x = x.view(-1, 12)
#         x = x.relu()
#         return x
    
#     def train(self, data_loader, test_loader, wape_call, epoch: int):
#         batch_bar = tqdm(data_loader, desc=f"Epoch {0}, loss={0:.5f}")
#         for i in range(epoch):
#             batch_bar.reset()
#             sum_loss = 0
#             for batch in data_loader:
#                 targets, _, _, _, vtrends = batch
#                 self.optimizer.zero_grad()
#                 loss = self.forward(
#                     vtrends, targets
#                 )
#                 loss.backward()
#                 sum_loss += loss
#                 self.optimizer.step()
#                 batch_bar.update()
#                 batch_bar.refresh()

#             batch_bar.set_description(f"Epoch {i}, loss={sum_loss:.5f}")

#             targets, _, _, _, vtrends = test_loader.dataset[:]
#             outputs = self.predict(vtrends)
#             mlflow.log_metric("val_wape", wape_call(targets, outputs), step=i)

#             targets, _, _, _, vtrends = data_loader.dataset[:]
#             outputs = self.predict(vtrends)
#             mlflow.log_metric("train_wape", wape_call(targets, outputs), step=i)

In [8]:
mlflow.start_run()

<ActiveRun: >

In [10]:
model = TransformerPoolingForecast(4)
model.predict(vtrends).shape

torch.Size([32, 12])

In [8]:
# model.train(train_loader, test_loader, misc_utils.cal_wape, 50)

In [5]:
device = "mps"

In [6]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from fairseq.optim.adafactor import Adafactor


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=52):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TimeDistributed(nn.Module):
    # Takes any module and stacks the time dimension with the batch dimenison of inputs before applying the module
    # Insipired from https://keras.io/api/layers/recurrent_layers/time_distributed/
    # https://discuss.pytorch.org/t/any-pytorch-function-can-work-as-keras-timedistributed/1346/4
    def __init__(self, module, batch_first=True):
        super(TimeDistributed, self).__init__()
        self.module = module # Can be any layer we wish to apply like Linear, Conv etc
        self.batch_first = batch_first

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)

        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, x.size(-1))  

        y = self.module(x_reshape)

        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)

        return y

class FusionNetwork(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, use_img, use_text, dropout=0.2):
        super(FusionNetwork, self).__init__()
        
        self.img_pool = nn.AdaptiveAvgPool2d((1,1))
        self.img_linear = nn.Linear(2048, embedding_dim)
        self.use_img = use_img
        self.use_text = use_text
        input_dim = embedding_dim + (embedding_dim*use_img) + (embedding_dim*use_text)
        self.feature_fusion = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, input_dim, bias=False),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(input_dim, hidden_dim)
        )

    def forward(self, img_encoding, text_encoding, dummy_encoding):
        # Fuse static features together

        # Build input
        decoder_inputs = []
        if self.use_img == 1:
            pooled_img = self.img_pool(img_encoding)
            condensed_img = self.img_linear(pooled_img.flatten(1))
            decoder_inputs.append(condensed_img) 
        if self.use_text == 1:
            decoder_inputs.append(text_encoding) 
        decoder_inputs.append(dummy_encoding)
        concat_features = torch.cat(decoder_inputs, dim=1)

        final = self.feature_fusion(concat_features)
        # final = self.feature_fusion(dummy_encoding)

        return final

class GTrendEmbedder(nn.Module):
    def __init__(self, forecast_horizon, embedding_dim, use_mask, trend_len, num_trends,  gpu_num):
        super().__init__()
        self.forecast_horizon = forecast_horizon
        self.input_linear = TimeDistributed(nn.Linear(num_trends, embedding_dim))
        self.pos_embedding = PositionalEncoding(embedding_dim, max_len=trend_len)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=4, dropout=0.2)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.use_mask = use_mask
        self.gpu_num = gpu_num

    def _generate_encoder_mask(self, size, forecast_horizon):
        mask = torch.zeros((size, size))
        split = math.gcd(size, forecast_horizon)
        for i in range(0, size, split):
            mask[i:i+split, i:i+split] = 1
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(device)
        return mask
    
    def _generate_square_subsequent_mask(self, size):
        mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(device)
        return mask

    def forward(self, gtrends):
        gtrend_emb = self.input_linear(gtrends.permute(0,2,1))
        gtrend_emb = self.pos_embedding(gtrend_emb.permute(1,0,2))
        input_mask = self._generate_encoder_mask(gtrend_emb.shape[0], self.forecast_horizon)
        if self.use_mask == 1:
            gtrend_emb = self.encoder(gtrend_emb, input_mask)
        else:
            gtrend_emb = self.encoder(gtrend_emb)
        return gtrend_emb
    
    
class DummyEmbedder(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.day_embedding = nn.Linear(1, embedding_dim)
        self.week_embedding = nn.Linear(1, embedding_dim)
        self.month_embedding = nn.Linear(1, embedding_dim)
        self.year_embedding = nn.Linear(1, embedding_dim)
        self.dummy_fusion = nn.Linear(embedding_dim*4, embedding_dim)
        self.dropout = nn.Dropout(0.2)


    def forward(self, temporal_features):
        # Temporal dummy variables (day, week, month, year)
        d, w, m, y = temporal_features[:, 0].unsqueeze(1), temporal_features[:, 1].unsqueeze(1), \
            temporal_features[:, 2].unsqueeze(1), temporal_features[:, 3].unsqueeze(1)
        d_emb, w_emb, m_emb, y_emb = self.day_embedding(d), self.week_embedding(w), self.month_embedding(m), self.year_embedding(y)
        temporal_embeddings = self.dummy_fusion(torch.cat([d_emb, w_emb, m_emb, y_emb], dim=1))
        temporal_embeddings = self.dropout(temporal_embeddings)

        return temporal_embeddings

class TransformerDecoderLayer(nn.Module):

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(TransformerDecoderLayer, self).__init__()
        
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)

        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.activation = F.relu

    def __setstate__(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super(TransformerDecoderLayer, self).__setstate__(state)

    def forward(self, tgt, memory, tgt_mask = None, memory_mask = None, tgt_key_padding_mask = None, 
            memory_key_padding_mask = None):

        tgt2, attn_weights = self.multihead_attn(tgt, memory, memory)
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt, attn_weights


In [10]:
class GTM(pl.LightningModule):
    def __init__(self, embedding_dim, hidden_dim, output_dim, num_heads, num_layers, trend_len, num_trends, use_encoder_mask=1, autoregressive=False):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.output_len = output_dim
        self.use_encoder_mask = use_encoder_mask
        self.autoregressive = autoregressive
        self.save_hyperparameters()

         # Encoder
        self.dummy_encoder = DummyEmbedder(embedding_dim)
        self.gtrend_encoder = GTrendEmbedder(output_dim, hidden_dim, use_encoder_mask, trend_len, num_trends, 0)
        self.static_feature_encoder = FusionNetwork(embedding_dim, hidden_dim, False, False)

        # Decoder
        self.decoder_linear = TimeDistributed(nn.Linear(1, hidden_dim))
        decoder_layer = TransformerDecoderLayer(d_model=self.hidden_dim, nhead=num_heads, \
                                                dim_feedforward=self.hidden_dim * 4, dropout=0.1)
        
        if self.autoregressive: self.pos_encoder = PositionalEncoding(hidden_dim, max_len=12)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(hidden_dim, self.output_len if not self.autoregressive else 1),
            nn.Dropout(0.2)
        )
        self.val_outputs = []
        self.val_targets = []


    def _generate_square_subsequent_mask(self, size):
        mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(device)
        return mask

    def forward(self, temporals, trends):
        # Encode features and get inputs
        # img_encoding = self.image_encoder(images)
        dummy_encoding = self.dummy_encoder(temporals)
        # text_encoding = self.text_encoder(category, color, fabric)
        gtrend_encoding = self.gtrend_encoder(trends)

        # Fuse static features together
        static_feature_fusion = self.static_feature_encoder(None, None, dummy_encoding)

        if self.autoregressive == 1:
            # Decode
            tgt = torch.zeros(self.output_len, gtrend_encoding.shape[1], gtrend_encoding.shape[-1]).to(device)
            tgt[0] = static_feature_fusion
            tgt = self.pos_encoder(tgt)
            tgt_mask = self._generate_square_subsequent_mask(self.output_len)
            memory = gtrend_encoding
            decoder_out, attn_weights = self.decoder(tgt, memory, tgt_mask)
            forecast = self.decoder_fc(decoder_out)
        else:
            # Decode (generatively/non-autoregressively)
            tgt = static_feature_fusion.unsqueeze(0)
            memory = gtrend_encoding
            decoder_out, attn_weights = self.decoder(tgt, memory)
            forecast = self.decoder_fc(decoder_out)

        return forecast.view(-1, self.output_len), attn_weights

    def configure_optimizers(self):
        optimizer = Adafactor(self.parameters(),scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
        return [optimizer]


    def training_step(self, train_batch, batch_idx):
        # item_sales, category, color, fabric, temporal_features, gtrends, images = train_batch 
        targets, attrs, temporals, gtrends, vtrends = train_batch
        outs, _ = self.forward(temporals, vtrends)
        loss = F.mse_loss(targets, outs)
        return loss


    def validation_step(self, test_batch, batch_idx):
        targets, attrs, temporals, gtrends, vtrends = test_batch
        outs, _ = self.forward(temporals, vtrends)
        self.val_outputs.extend(outs)
        self.val_targets.extend(targets)


    def on_validation_epoch_end(self) -> None:
        outs = torch.stack(self.val_outputs)
        targets = torch.stack(self.val_targets)

        print("val_loss", F.mse_loss(targets, outs))
        print("val_wape", misc_utils.cal_wape(targets, outs))
        
        self.val_outputs.clear()
        self.val_targets.clear()

    # def validation_epoch_end(self, val_step_outputs):
    #     item_sales, forecasted_sales = [x[0] for x in val_step_outputs], [x[1] for x in val_step_outputs]
    #     item_sales, forecasted_sales = torch.stack(item_sales), torch.stack(forecasted_sales)
    #     rescaled_item_sales, rescaled_forecasted_sales = item_sales*1065, forecasted_sales*1065 # 1065 is the normalization factor (max of the sales of the training set)
    #     loss = F.mse_loss(item_sales, forecasted_sales.squeeze())
    #     mae = F.l1_loss(rescaled_item_sales, rescaled_forecasted_sales)
        

    #     print('Validation MAE:', mae.detach().cpu().numpy(), 'LR:', self.optimizers().param_groups[0]['lr'])

In [11]:
model = GTM(
    embedding_dim=32, hidden_dim=64, output_dim=12, num_heads=4, num_layers=1, trend_len=196, num_trends=4,
    # autoregressive=True, use_encoder_mask=True
)

In [13]:
trainer = pl.Trainer(accelerator=device, devices=1, max_epochs=100, check_val_every_n_epoch=1)
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=test_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                   | Type               | Params
--------------------------------------------------------------
0 | dummy_encoder          | DummyEmbedder      | 4.4 K 
1 | gtrend_encoder         | GTrendEmbedder     | 562 K 
2 | static_feature_encoder | FusionNetwork      | 68.8 K
3 | decoder_linear         | TimeDistributed    | 128   
4 | decoder                | TransformerDecoder | 50.0 K
5 | decoder_fc             | Sequential         | 780   
--------------------------------------------------------------
686 K     Trainable params
0         Non-trainable params
686 K     Total params
2.747     Total estimated model params size (MB)


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  4.21it/s]val_loss tensor(0.0040, device='mps:0')
val_wape 64.131
Epoch 0: 100%|██████████| 64/64 [00:31<00:00,  2.01it/s, v_num=13]         val_loss tensor(0.0026, device='mps:0')
val_wape 58.607
Epoch 1: 100%|██████████| 64/64 [00:28<00:00,  2.25it/s, v_num=13]val_loss tensor(0.0026, device='mps:0')
val_wape 58.503
Epoch 2: 100%|██████████| 64/64 [00:28<00:00,  2.28it/s, v_num=13]val_loss tensor(0.0026, device='mps:0')
val_wape 58.552
Epoch 3: 100%|██████████| 64/64 [00:28<00:00,  2.26it/s, v_num=13]val_loss tensor(0.0026, device='mps:0')
val_wape 58.366
Epoch 4: 100%|██████████| 64/64 [00:28<00:00,  2.27it/s, v_num=13]val_loss tensor(0.0027, device='mps:0')
val_wape 58.334
Epoch 5: 100%|██████████| 64/64 [00:28<00:00,  2.26it/s, v_num=13]val_loss tensor(0.0026, device='mps:0')
val_wape 58.258
Epoch 6: 100%|██████████| 64/64 [00:28<00:00,  2.26it/s, v_num=13]val_loss tensor(0.0026, device='mps:0')
val_wape 58.541
Epoch 

/Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
